In [1]:
import os
from fastai.vision.all import *
import pandas as pd
from sklearn.model_selection import StratifiedGroupKFold
import wandb
import warnings
from config import WANDB_PROJECT, ENTITY, RAW_DATA_AT
warnings.filterwarnings('ignore')

In [3]:
run = wandb.init(project=WANDB_PROJECT, entity=ENTITY, job_type="data_split")

wandb: Currently logged in as: darek. Use `wandb login --relogin` to force relogin


In [4]:
raw_data_at = run.use_artifact(f'{RAW_DATA_AT}:latest')
path = Path(raw_data_at.download())

wandb: Downloading large artifact bdd_sample_1k:latest, 124.77MB. 2001 files... 
wandb:   2001 of 2001 files downloaded.  
Done. 0:0:0.1


In [5]:
path.ls()

(#3) [Path('artifacts/bdd_sample_1k:v0/labels'),Path('artifacts/bdd_sample_1k:v0/LICENSE.txt'),Path('artifacts/bdd_sample_1k:v0/images')]

In [6]:
fnames = os.listdir(path/'images')
groups = [s.split('-')[0] for s in fnames]

In [7]:
orig_eda = run.use_artifact("run-1xxf9tvp-eda_table:v0")
orig_eda_table = orig_eda.get("eda_table")

wandb: Downloading large artifact run-1xxf9tvp-eda_table:v0, 732.03MB. 2002 files... 
wandb:   2002 of 2002 files downloaded.  
Done. 0:0:0.1


In [8]:
y = orig_eda_table.get_column('train')

In [9]:
df = pd.DataFrame()
df['File_Name'] = fnames
df['fold'] = -1

In [11]:
cv = StratifiedGroupKFold(n_splits=10)
for i, (train_idxs, test_idxs) in enumerate(cv.split(fnames, y, groups)):
    df.loc[test_idxs, ['fold']] = i

In [12]:
df['Stage'] = 'train'
df.loc[df.fold == 0, ['Stage']] = 'test'
df.loc[df.fold == 1, ['Stage']] = 'valid'
del df['fold']
df.Stage.value_counts()

train    800
valid    100
test     100
Name: Stage, dtype: int64

In [13]:
df.to_csv('data_split.csv', index=False)

In [14]:
processed_data_at = wandb.Artifact("bdd_sample_1k_split", type="split_data")

In [15]:
processed_data_at.add_file('data_split.csv')
processed_data_at.add_dir(path)

wandb: Adding directory to artifact (./artifacts/bdd_sample_1k:v0)... Done. 0.2s


In [16]:
data_split_table = wandb.Table(dataframe=df[['File_Name', 'Stage']])

In [17]:
join_table = wandb.JoinedTable(orig_eda_table, data_split_table, "File_Name")

In [18]:
processed_data_at.add(join_table, "eda_table_data_split")

<ManifestEntry digest: ik+42Usji16Cc30/Hkw5fg==>

In [19]:
run.log_artifact(processed_data_at)
run.finish()

wandb: Network error (ConnectTimeout), entering retry loop.
wandb: Network error (ConnectTimeout), entering retry loop.
